
# To utilize both the ipywidgets and ipyleaflet functions you must first ensure that you have them installed. (see https://ipyleaflet.readthedocs.io/en/latest/installation.html & https://ipywidgets.readthedocs.io/en/latest/user_install.html. Then use the import function for ipywidgets to get a search box that collects user input in the form of a zipcode 

In [136]:
import ipywidgets as w
import requests

zipcode_input = w.Text(
    value='',
    placeholder='',
    description='Zipcode:',
    disabled=False
)
zipcode_input

Text(value='', description='Zipcode:', placeholder='')

# Use import function for geocoder that searches a map
# Use import function to obtain raw json data from https://vaccinefinder.org/
# Define the get_data_from_api function that passes a zipcode no more than 5 numbers into it which also will request an api call using https://api.findertools.org/api/swagger/
# Parse through the raw json data to search for specific locations based on user provided zipcode and lists those vaccination locations and coordinates for that zipcode

In [143]:
import geocoder
import json
from ipyleaflet import Map, Marker, MarkerCluster, FullScreenControl, AwesomeIcon

# list to hold lat and lng for centering the map 
map_centre = []

# list to hold markers for marker cluster
markers = []

def get_data_from_api(zipcode):
    if (len(zipcode) < 5):
        print('not valid')
    else:
        count = 0
        g = geocoder.osm(zipcode)
        
        # search radius
        distance_search = 10
        # lat associatd with input (zipcode)
        lat = g.latlng[0]
        # append lat to map_centre list
        map_centre.append(lat)
        # lng associatd with input (zipcode)
        lng = g.latlng[1]
        # append lng to map_centre list
        map_centre.append(lng)
        
        # create a string using the API web-address, lat, lng and search radius and save to request_str
        request_str = 'https://api.findertools.org/api/v1/public/providers/vaccinefinder/locations/?localization=' + str(lng) + ',' + str(lat) + '&distance=' + str(distance_search)
        # use requests to make the API call and save output to response
        response = requests.get(request_str)
        # convert response to a JSON object and save in parsed
        parsed = json.loads(response.text)
        
        # iterate over data in parsed['features']
        for features in parsed["features"]:
            # only use the first 50 entries returned from the API call
            if count == 50:
                break
            # save location's address and name in respective variables
            address = features["properties"]["address"]
            name = features["properties"]["name"]
            
            # API retunred lng, lat (should be lat THEN lng) so this must be reversed
            lat = features["geometry"]["coordinates"][1]
            long = features["geometry"]["coordinates"][0]
            
            # check that each entry is not null
            if(features["geometry"]["coordinates"] != None):
                # create a marker using the valid entry
                marker = Marker(location = [lat, long], title = name + ' at ' + address, icon = health_icon)
                # append the marker to the list (markers)
                markers.append(marker)
                # increment count by 1
                count = count + 1

# call the function
w.interactive_output(get_data_from_api, { 'zipcode': zipcode_input })


Output()

# create and render map using data from the get_data_from_api function

In [144]:

# create a marker cluster using the list of markers created in the get_data_from_api function
marker_cluster = MarkerCluster(markers = (markers))

# icon for map markers
health_icon = AwesomeIcon(
    name='fa-medkit',
    marker_color='red',
    icon_color='white',
    spin=True
)

# create map and set the center to be coordinates stored in map_centre 
map = Map(center = map_centre, zoom = 10)

# add cluster and full screen control to the map
map.add_layer(marker_cluster)
map.add_control(FullScreenControl())

# display map
map

Map(center=[39.952790756651055, -82.80129294314598], controls=(ZoomControl(options=['position', 'zoom_in_text'…